# This version uses random splitting of data, for better results, dat ashould be splited in linear manner. Version two doe that.

In [1]:
#Importing usefull libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#importing algorithms
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

%matplotlib inline

In [3]:
#Reading Data
data = pd.read_excel('ETR-259-TRU 2 Hardware and Operating Conditions Matrix 2.3 V4 (Rev 2).xlsx', 'LinearEngineMap', index_col=False)
data.head()

,N2 Temp. [°C],HEF Temp. [°C],Engine Speed [RPM],N2 Pressure [barg],Break Power [W],BSFC [Wh/kg],FMEP,Unnamed: 7,Excel range,Unnamed: 9
0,15.0,15.0,800.0,30.0,NaN,NaN,NaN,NaN,H15:H31,H15:H31
1,15.0,15.0,800.0,37.5,NaN,NaN,NaN,NaN,H32:H48,H32:H48
2,15.0,15.0,800.0,45.0,NaN,NaN,NaN,NaN,H49:H65,H49:H65
3,15.0,15.0,900.0,30.0,NaN,NaN,NaN,NaN,H66:H82,H66:H82
4,15.0,15.0,900.0,37.5,NaN,NaN,NaN,NaN,H83:H99,H83:H99


In [4]:
#dropping unusable columns
df = data.drop(["Unnamed: 7", "Excel range","Unnamed: 9"], axis=1)
df.head()

,N2 Temp. [°C],HEF Temp. [°C],Engine Speed [RPM],N2 Pressure [barg],Break Power [W],BSFC [Wh/kg],FMEP
0,15.0,15.0,800.0,30.0,NaN,NaN,NaN
1,15.0,15.0,800.0,37.5,NaN,NaN,NaN
2,15.0,15.0,800.0,45.0,NaN,NaN,NaN
3,15.0,15.0,900.0,30.0,NaN,NaN,NaN
4,15.0,15.0,900.0,37.5,NaN,NaN,NaN


## I want to drop also the test points which are not within the  nxn  smaller 4D maps
* HEF T - 15, N2 T -25 and 35
* N2 T - 15, HEF T -25 and 35

In [5]:
#dropping NaN values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 18 to 323
Data columns (total 7 columns):
N2 Temp. [°C]         142 non-null float64
HEF Temp. [°C]        142 non-null float64
Engine Speed [RPM]    142 non-null float64
N2 Pressure [barg]    142 non-null float64
Break Power [W]       142 non-null float64
BSFC [Wh/kg]          142 non-null float64
FMEP                  142 non-null float64
dtypes: float64(7)
memory usage: 8.9 KB


In [6]:
df.head()

,N2 Temp. [°C],HEF Temp. [°C],Engine Speed [RPM],N2 Pressure [barg],Break Power [W],BSFC [Wh/kg],FMEP
18,15.0,25.0,800.0,30.0,2.664000,37.420000,1.044000
19,15.0,25.0,800.0,37.5,3.562222,39.605556,1.171111
20,15.0,25.0,800.0,45.0,4.498000,41.088000,1.324000
21,15.0,25.0,900.0,30.0,2.760000,36.610000,1.102500
22,15.0,25.0,900.0,37.5,3.750833,38.986667,1.239167


In [7]:
df.columns

Index(['N2 Temp. [°C]', 'HEF Temp. [°C]', 'Engine Speed [RPM]',
       'N2 Pressure [barg]', 'Break Power [W]', 'BSFC [Wh/kg]', 'FMEP'],
      dtype='object')

## df is all the test data

In [8]:
df = df[~((df["N2 Temp. [°C]"] == 15.0) & ((df["HEF Temp. [°C]"] == 25.0) | (df["HEF Temp. [°C]"] == 35.0)))]
df = df[~((df["HEF Temp. [°C]"] == 15.0) & ((df["N2 Temp. [°C]"] == 25.0) | (df["N2 Temp. [°C]"] == 35.0)))]

## df2 to df5 is 5 off 3x3 for the linear model validation

In [9]:
df2 = df[((df["N2 Temp. [°C]"] == 20.0) & (df["HEF Temp. [°C]"] == 20.0))]
df3 = df[((df["N2 Temp. [°C]"] == 40.0) & (df["HEF Temp. [°C]"] == 20.0))]
df4 = df[((df["N2 Temp. [°C]"] == 30.0) & (df["HEF Temp. [°C]"] == 30.0))]
df5 = df[((df["N2 Temp. [°C]"] == 20.0) & (df["HEF Temp. [°C]"] == 40.0))]
df6 = df[((df["N2 Temp. [°C]"] == 40.0) & (df["HEF Temp. [°C]"] == 40.0))]

In [10]:
df3

,N2 Temp. [°C],HEF Temp. [°C],Engine Speed [RPM],N2 Pressure [barg],Break Power [W],BSFC [Wh/kg],FMEP
279,40.0,20.0,800.0,30.0,2.666667,37.881667,1.036667
280,40.0,20.0,800.0,37.5,3.549000,39.826000,1.191000
281,40.0,20.0,800.0,45.0,4.552000,41.826000,1.310000
282,40.0,20.0,900.0,30.0,2.770000,36.620000,1.146000
283,40.0,20.0,900.0,37.5,3.740909,39.034545,1.286364
284,40.0,20.0,900.0,45.0,4.746667,40.815000,1.426667
285,40.0,20.0,1000.0,30.0,3.018333,36.730000,1.196667
286,40.0,20.0,1000.0,37.5,4.025000,39.127500,1.325000
287,40.0,20.0,1000.0,45.0,5.147500,40.770000,1.460000


In [11]:
df_short1 = [df2, df3, df4, df5, df6]
df_short = pd.concat(df_short1)

In [12]:
df_short

,N2 Temp. [°C],HEF Temp. [°C],Engine Speed [RPM],N2 Pressure [barg],Break Power [W],BSFC [Wh/kg],FMEP
63,20.0,20.0,800.0,30.0,2.650000,37.112500,1.060000
64,20.0,20.0,800.0,37.5,3.568182,39.229091,1.178182
65,20.0,20.0,800.0,45.0,4.518333,40.583333,1.315000
66,20.0,20.0,900.0,30.0,2.767500,36.185000,1.090000
67,20.0,20.0,900.0,37.5,3.784000,38.537000,1.217000
68,20.0,20.0,900.0,45.0,4.747500,40.160000,1.353333
69,20.0,20.0,1000.0,30.0,2.993333,36.028333,1.165000
70,20.0,20.0,1000.0,37.5,4.045455,38.280000,1.309091
71,20.0,20.0,1000.0,45.0,5.120000,39.860000,1.456667
279,40.0,20.0,800.0,30.0,2.666667,37.881667,1.036667


# Predictions for the first case
## First case - getting the 117 tests and predicting on them

In [13]:
#randomly splitting data for training ang testing
msk = np.random.rand(len(df)) <=0.3

train=df[msk]
test=df[~msk]

In [14]:
#asigning new variables for predictions, dropping the variable which I want to predict
x_train = train.drop(["Break Power [W]","FMEP","BSFC [Wh/kg]"], axis = 1)
y_train = train["Break Power [W]"]
x_test = test.drop(["Break Power [W]","FMEP","BSFC [Wh/kg]"], axis=1)
x_train.shape, y_train.shape,x_test.shape

((39, 4), (39,), (78, 4))

In [15]:
from sklearn.linear_model import LinearRegression 

lr = LinearRegression()
lr.fit(x_train, y_train)
Y_pred = lr.predict(x_test)
difference = 100*(((Y_pred)-test["Break Power [W]"])/(Y_pred))

predicted_power = pd.DataFrame({
        "HEF Temp. [°C]": test["HEF Temp. [°C]"],
        "Engine Speed [RPM]": test["Engine Speed [RPM]"],
        "N2 Pressure [barg]": test["N2 Pressure [barg]"],
        "BSFC [Wh/kg]": test["BSFC [Wh/kg]"],
        "N2 Temp. [°C]":test["N2 Temp. [°C]"],
        "Break Power [W]":test["Break Power [W]"],
        "Break Power [W]_pred": Y_pred,
        "Difference": difference
    })

#predicted_power.to_csv('LinearRegression.csv', index=False)
print("The maximum percentage difference is:", difference.max())
print ("The min percentage difference is:", difference.min())
predicted_power.to_csv('Predictions_large.csv')

The maximum percentage difference is: 4.472105483298867
The min percentage difference is: -3.7835531966698603


### Second case when we take only the corner and middle 3x3 test

In [16]:
#randomly splitting data for training ang testing
msk2 = np.random.rand(len(df_short)) <=0.3

train2=df_short[msk2]
test2=df_short[~msk2]

In [17]:
#asigning new variables for predictions, dropping the variable which I want to predict
x_train2 = train2.drop(["Break Power [W]","FMEP","BSFC [Wh/kg]"], axis = 1)
y_train2 = train2["Break Power [W]"]
x_test2 = test2.drop(["Break Power [W]","FMEP","BSFC [Wh/kg]"], axis=1)
x_train2.shape, y_train2.shape,x_test2.shape

((19, 4), (19,), (26, 4))

In [18]:
lr2 = LinearRegression()
lr2.fit(x_train2, y_train2)
Y_pred2 = lr2.predict(x_test2)
difference2 = 100*(((Y_pred2)-test2["Break Power [W]"])/(Y_pred2))

predicted_power2 = pd.DataFrame({
        "HEF Temp. [°C]": test2["HEF Temp. [°C]"],
        "Engine Speed [RPM]": test2["Engine Speed [RPM]"],
        "N2 Pressure [barg]": test2["N2 Pressure [barg]"],
        "BSFC [Wh/kg]": test2["BSFC [Wh/kg]"],
        "N2 Temp. [°C]":test2["N2 Temp. [°C]"],
        "Break Power [W]":test2["Break Power [W]"],
        "Break Power [W]_pred": Y_pred2,
        "Difference": difference2
    })

#predicted_power.to_csv('LinearRegression.csv', index=False)
print("The maximum percentage difference is:", difference2.max())
print ("The min percentage difference is:", difference2.min())
predicted_power2.to_csv('Predictions_small.csv')

The maximum percentage difference is: 3.878846686678001
The min percentage difference is: -6.650722384531277
